In [ ]:
# EXAMPLE 2 - Create linear regression models for all 60 pole transformers.
# Read PI data via PI OLEDB Enterprise using the adodbapi package to establish an OLEDB connection.

# Import Python packages.
import adodbapi as ado                        # OLEDB support for accessing PI SQL Client - import AF/PI data.
import pandas as pd                           # Dataframe support.
from sklearn.linear_model import LinearRegression # Linear regression model from the scikit-learn package.

# Connect to "Distribution Network Lab" model in AF through the PI OLEDB Enterprise.
PI_connection = ado.connect("Provider=PIOLEDBEnt; Integrated Security=SSPI; Data Source=PISRV01;Command Timeout=0;")

# Create a cursor object to access the data serer for the "Distribution Network Lab" database in AF.
PI_cursor = PI_connection.cursor()

# Select the entire table using the view configured in PI SQL Commander.
PI_cursor.execute('SELECT * FROM [Distribution Network].[Asset].[Pole Transformer Loads]')

# Extract first row, index=0, to get column names for use as dataframe headers 
columnNames = [ x[0] for x in PI_cursor.description]
print("PI View column Names:\n\n", columnNames)

# Unpack the cursor rows into a datafraem.
poleTransformerLoads = pd.DataFrame([dict(zip(columnNames, row)) for row in PI_cursor], columns=columnNames)

# Take a look to see if everythking worked.
poleTransformerLoads.head()

In [ ]:
# Rename some columns with shorter names to make them easier to work with.
poleTransformerLoads.rename(columns = {'Name' :'Transformer'}, inplace = True)
poleTransformerLoads.rename(columns = {'Wh Delivered Load':'Wh Load'}, inplace = True )
poleTransformerLoads.rename(columns = {'Wh Delivered Load - 14d':'Wh Load-14d'}, inplace = True )
poleTransformerLoads.rename(columns = {'Wh Delivered Load - 7d':'Wh Load-7d'}, inplace = True )
poleTransformerLoads.rename(columns = {'Ambient Temperature':'Temperature'}, inplace = True )
poleTransformerLoads.rename(columns = {'Relative Humidity':'Humidity'}, inplace = True )
poleTransformerLoads.rename(columns = {'Wind Speed':'Wind'}, inplace = True )

# Define second dataframe with just data needed for our modelling.
modellingData = poleTransformerLoads[['Transformer', 'Timestamp', 'Temperature', 'Humidity',
                    'Wind','Wh Load', 'Wh Load-7d', 'Wh Load-14d']]

# Peek at the first five rows to make sure things look right.
modellingData.head()

In [ ]:
# In order to analyze transformers individually, we need to set the datframe's index to the "Transformer" column. 
modellingData = modellingData.set_index("Transformer", drop=False)

# Take a look, see the difference?
modellingData.head()

In [ ]:
# Use our existing PI View connectiont to select just the transformer names from the view configured in
# PI SQL Commander.
PI_cursor.execute('SELECT DISTINCT Name FROM [Distribution Network].[Asset].[Pole Transformer Loads]')

# Create a list for the transformer names.
transformerNames = []

# Populate the list
for row in PI_cursor:
    transformerNames.append(row[0])

# Show tranfomer names list.
print(transformerNames)

In [ ]:
# Using the "adodbapi" package, to connect to the "Predictive Equations" MS SQL.

# Set connection parameters.
con_string = 'DRIVER={SQL Server};SERVER=PISRV01;DATABASE=PIWorld;Trusted_Connection=Yes;'

# Connect to "Distribution Network Lab" model in AF through the PI SQL Client.
SQL_connection = ado.connect(con_string)

# Create a cursor object to access the data serer for the "Distribution Network Lab" database in AF.
SQL_cursor = SQL_connection.cursor()

# Create linear regression object from the "sklearn" package we imported earlier.
LinReg = LinearRegression()

# Looping through the trasformer list, Perform linear regression on each transformer.
for transformer in transformerNames:
    
    # Create dataframe for one transformer.
    transformerData = modellingData.loc[transformer,:]
    
    # Perform linear regression fit
    LinReg.fit(transformerData[["Wh Load-7d","Wh Load-14d","Temperature","Humidity"]],transformerData["Wh Load"])
    
    # Update asset ID value with the name of this transformer.
    asset_id = "'"+transformer+"'" 
    
    # Print equation.
    print(transformer, "Eq:\n", LinReg.coef_[0], "*","'Wh Delivered Load - 7d' + ",LinReg.coef_[1],
          "*","'Wh Delivered Load - 14d' + ", LinReg.coef_[2], "*", "'Ambient Temperature' +",
          LinReg.coef_[3], "*","'Relative Humidity' +(",LinReg.intercept_, ")" )
    
    ## INSERT and UPDATE queries to load table for the first time or update an existing one.
    # Construct query to add this transfomer's model coefficients.
    insert_query = f'INSERT [Predictive Equations] ([Asset ID], Coefficient_0, Coefficient_1, Coefficient_2, Coefficient_3, Coefficient_4, Intercept) values({asset_id}, {LinReg.coef_[0]}, {LinReg.coef_[1]}, {LinReg.coef_[2]}, {LinReg.coef_[3]}, null, {LinReg.intercept_})'

    update_query = f'UPDATE [Predictive Equations] SET Coefficient_0={LinReg.coef_[0]}, Coefficient_1={LinReg.coef_[1]},Coefficient_2={LinReg.coef_[2]},Coefficient_3={LinReg.coef_[3]},Intercept={LinReg.intercept_} WHERE [Asset ID]= {asset_id}'
    
    # Insert this record into the "Predictive Equations" table.
    SQL_cursor.execute(update_query)
    
# Commit the queries to write the data into SQL.
SQL_connection.commit()

# Close database connections.
SQL_connection.close()
PI_connection.close()